In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import datetime, itertools, zipfile, os, time, psutil, gc, sys
from datetime import date, timedelta
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelEncoder
sys.path.append('../input/my-temp-files/')
    
import alml2 as alml

RS = alml.rs(0)

SUBMIT = False
SUBMIT = True

MODELX = 'A' #A - best LB, B - alternative, C - best CV

#3.681205
#3.677905 - 3.722
#3.666167 - 3.708
#3.662555 - 3.707
#3.655946 - 3.699
#3.655045 - 3.699 (little bit better)
#3.653728 - 3.697
#3.650593 - 3.692
#3.650871 - 3.689
#3.646806 - 3.687 (m_h)
#3.646362 - 3.687 (-2 feats)
#3.644921 - 3.684 (hist m1 feats)
#3.644489 - 3.684 better (new m1 feat)
#3.643781 - 3.684 better (add aggs month_diff) <<<
#3.642602 - 3.685 - C mod
#3.642493 - 3.683 - min/max dates to weeks
#3.641609 - 3.682 CB - feat 1-2 interaction (RS=1 3.642509)(RS=2 3.643232)

ROUNDS = 1000

def rmse(y_true, y_pred):
	score = np.sqrt(mean_squared_error(y_true, y_pred))
	return score

NROWS = None
NROWS_VAL = None
if 0:
	NROWS = 100000 #None 
	NROWS_VAL = 20000

MODELS = ['lgb']
#MODELS = ['lgb','rfr','lnr','lasso','ridge','abr','etr','gbr']
#MODELS = ['ridge']

MODELS2 = ['lgb']
LEVEL = 1 

BAGS = 0 if SUBMIT else 0
FOLDS = 5
EARLY_STOPPING = 100 #100

if LEVEL == 2:
	FOLDS = 0
	MODELS = MODELS2

# XGB
params1 = {}
params1['objective'] = 'binary:logistic'		#'binary:logistic'
params1['eval_metric'] = 'auc'			#'logloss'
params1['eta'] = 0.04
params1['max_depth'] = 7
params1['silent'] = 1
params1['subsample'] = 0.8
params1['subsample_bytree'] = 0.7
#params1['subsample_bylevel'] = 0.7
params1['num_rounds'] = 30 #100

#LightGBM
params2 = {}
params2['objective'] = 'regression' #'regression'
params2['metric'] = 'rmse'		#'multi_logloss', 'binary_logloss'
params2['boosting'] = 'gbdt' #'gbdt'
params2['learning_rate'] = 0.03 #0.2
params2['verbose'] = -1
#params2['num_class'] = 6
params2['num_leaves'] = 63 # 1:2 2:4 3:8 4:16 5:32 6:64 7:128 8:256
params2['bagging_fraction'] = 0.8
params2['bagging_freq'] = 1
params2['bagging_seed'] = RS
params2['feature_fraction'] = 0.7
#params2['colsample_bytree'] = 0.05
params2['feature_fraction_seed'] = RS
#params2['scale_pos_weight'] = 10
#params2['max_bin'] = 300
params2['max_depth'] = 7 #6 #s-1
params2['min_child_samples'] = 90
#params2['min_gain_to_split'] = 0.5
#params2['reg_lambda'] = 0.1
params2['reg_alpha'] = 0.1
params2['num_rounds'] = ROUNDS
#params2['tree_learner'] = 'voting'
params2['feature_name'] = 'auto'
#params2['categorical_feature'] = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3"]
#params2['histogram_pool_size'] = 8000

params3 = {}
params3['n_jobs'] = 2
params3['n_estimators'] = 100
params3['max_depth'] = 5
params3['verbose'] = 0
params3['learning_rate'] = 0.1

if LEVEL == 2:
	#XGB
	ROUNDS = 980

CONFIG = []
if 'lgb' in MODELS:
	CONFIG.append({'model': 'lgb','params': params2,})
if 'etr' in MODELS:
	CONFIG.append({'model': 'etr','params': params3,})
if 'abr' in MODELS:
	CONFIG.append({'model': 'abr','params': params3,})
if 'gbr' in MODELS:
	CONFIG.append({'model': 'gbr','params': params3,})
if 'xgb' in MODELS:
	CONFIG.append({'model': 'xgb','params': params1,})
if 'rfr' in MODELS:
	CONFIG.append({'model': 'rfr','params': params3,})
if 'lasso' in MODELS:
	CONFIG.append({'model': 'lasso','params': params3,})
if 'ridge' in MODELS:
	CONFIG.append({'model': 'ridge','params': params3,})
if 'lnr' in MODELS:
	CONFIG.append({'model': 'lnr','params': params3,})

print("Started {}".format("SUBMIT" if SUBMIT else "TEST"))
print(psutil.virtual_memory())
target = 'target'
scorefn = rmse
feats_not_to_use = ['card_id', target, 'first_active_month']
cat_vars = [] 

def procfeats(df):
	return df
	
def add_feat(df, num, name, cols=None):
	ft = pd.read_pickle('{}.pkl'.format(name))
	if cols is not None:
		ft = ft[['card_id'] + cols]
	ft.columns = ['f{}_{}'.format(num, x) if x != 'card_id' else x for x in ft.columns]
	df = df.merge(ft, how='left', on='card_id')
	print(num,'Added',ft.shape,'features, current:',df.shape)
	return df

def add_submodel(df, name, feat=None):
	usecols = None if feat is None else ['card_id']+feat
	cv = pd.read_csv('runs/cv_{}.csv'.format(name), usecols=usecols)
	te = pd.read_csv('runs/test_{}.csv'.format(name), usecols=usecols)
	if 0:
		cv = cv[cv['submodel_2a_size'] > 3]
		te = te[te['submodel_2a_size'] > 3]
	cvte = pd.concat([cv, te]).reset_index(drop=True) #[['card_id','submodel_2a_min']]
	df = df.merge(cvte, how='left', on='card_id')
	print('Got',name,'with cv:',cv.shape,'test:',te.shape,'total:',cvte.shape,'current:',df.shape)
	return df

df = pd.read_pickle('../input/my-temp-files/feats_0001.pkl')
df_train = df[df['target'] != -999][['card_id','target']]
df_test = df[df['target'] == -999][['card_id','target']]
pos = df_train.shape[0]
	
if 0: #most correlated features
	df['out'] = df['target'].apply(lambda x: 1 if x < -20 else 0)
	tmp = df.corr()
	tmpt = tmp['target'].sort_values()
	print(tmpt)
	tmpt = tmp['out'].sort_values()
	print(tmpt)
	exit()
	
print(df.dtypes)
cat_vars = [f for f in df.columns if df[f].dtype == 'object' and f not in feats_not_to_use]
print('Categorical:',cat_vars)

for col in cat_vars:
	lbl = LabelEncoder()
	df[col] = lbl.fit_transform(df[col].values.astype('str'))

if 0:
	df, _, ffeats = alml.fe_cat_to_ohe(df, None, cat_vars)
	df.drop(cat_vars, axis=1, inplace=True)
	
print(df.head())
	
x_train = df[:pos].reset_index(drop=True)
x_test = df[pos:].reset_index(drop=True)	
del df
gc.collect()

#x_train, x_test, _ = alml.proc_add_noise(x_train, x_test, on=['binary_preds'], frac=2)

#x_train, x_test, _ = alml.fe_num_means_by_cat(x_train, x_test, x_train[target].values, on=['feature_1'], stat_col='diff_dates_in_max', prefix='tgenc', n_folds=5, stratify=False, type='mean')
#x_train, x_test, _ = alml.fe_target_encoding(x_train, x_test, x_train[target].values, on=['feature_2'], stat_col=None, prefix='tgenc', n_folds=5, stratify=False, type='mean')
#x_train, x_test, _ = alml.fe_target_encoding(x_train, x_test, x_train[target].values, on=['feature_3'], stat_col=None, prefix='tgenc', n_folds=5, stratify=False, type='mean')
#feats_not_to_use.append('feature_1')

use = [x for x in x_train.columns.values if x not in feats_not_to_use]
x_train = x_train[use]
x_test = x_test[use]
train_y = df_train[target].values
#train_y = np.abs(df_train[target].values)
#train_y = np.log(np.exp2(df_train[target].values))

print('Training on',x_train.shape, train_y.shape ,'testing on',x_test.shape)	
params2['categorical_feature'] = [x for x in cat_vars if x in use]
print('Categorical:',params2['categorical_feature'])	

gc.collect()
print(psutil.virtual_memory())
if 0:
	alml.features_eliminator(x_train, train_y, CONFIG[0]['params'], seed=RS, bags=2, score_fn=scorefn, logname='features_eliminator.txt')
	print('elimination done')
	exit()
if 0:
	alml.features_eliminator2(x_train, train_y, CONFIG[0]['params'], seed=RS, folds=FOLDS, score_fn=scorefn, logname='features_eliminator.txt', early_stopping=EARLY_STOPPING)
	print('elimination done')
	exit()
STRATIFYBY = False #True
if STRATIFYBY:
	x_train['is_outlier'] = (train_y < -20).astype(int)
	print(x_train.head())
preds, preds_cv, models, preds_cv_cols = alml.ml(x_train, train_y, x_test, CONFIG, None, True, BAGS, FOLDS, score_fn=scorefn, stratify=STRATIFYBY, early_stopping=EARLY_STOPPING)
	
del x_train, train_y
gc.collect()

now = str(datetime.datetime.now().strftime("%m%d%H%M"))
tf = 'm1_preds_%s.' % now
	
if 1:
	df_test['target'] = preds
	df_test = df_test[['card_id','target']]
	print(df_test.head())
	print(df_test.tail())
	
	print('\nPreparing submission...')

	print("Writing output...")
	df_test.to_csv(tf + 'csv', index=False)

	print(df_test.describe())
	print(df_test.shape)

	df_train['preds'] = preds_cv
	
	print('\n----- biggest errors')
	df_train['error'] = (df_train['preds'] - df_train[target]) ** 2
	print(df_train.sort_values('error', ascending=False).head(10))
	#df_train.sort_values('error', ascending=False).head(300).to_csv('outliers.csv', index=False)
	
	print('\n----- lowest preds')
	print(df_train.sort_values('preds', ascending=True).head(20))
	
	print('Score full on',df_train.shape[0],'entries',rmse(df_train[target],df_train['preds']))

	df_train = df_train[['card_id','target','preds']]
	savename = 'cv_' + tf + 'csv'
	print('\nPreparing tosave',df_train.shape)
	print(df_train.head())
	df_train.to_csv(savename, index=False)
	print('CV saved to',savename)
	
print("\nDone.")